In [1]:
from flask import Flask
from flask import render_template, request, redirect, url_for

import pymongo
from pymongo import MongoClient
from bson.objectid import ObjectId

import datetime
from datetime import datetime, date, time

import pprint
from bson.errors import InvalidId

client=MongoClient()
db=client.weddings

In [2]:
def create_opt_info(more):
    opt_info = input ('Хотите что-то добавить? (y/n) ')
    if opt_info == 'y':
        type_info = input("Введите параметр (например, website): ")
        info = input('Введите информацию: ')
        more.append({type_info:info})
        create_opt_info(more)
    elif opt_info == 'n':
        return more
    else:
        print ('Вы ввели неподходящий ответ')

In [3]:
def to_create(coll):
    collection = db[coll]
    if coll == 'projects':
        print ('Заполните поля:')
        id_wedding = input('ID свадьбы: ')
        date = input ('Дата гг-мм-дд. (оставьте пустым если хотите поставить дату заполнения): ')
        if date == '':
            date = datetime.today().strftime('%Y-%m-%d')
        type_work = input('Тип работы: ')
        client_comment = input('Комментарий заказчика: ')
        collection.insert_one({"id_wedding":id_wedding, "date":date, "type_work":type_work, "client_comment":client_comment})
        print('done')
    if coll == 'clients':
        print ('Сначала заполните обязательные поля: ')
        name_client = input('Название компании: ')
        date_contract = input('Дата начала контракта гг-мм-дд (оставьте пустым если хотите поставить дату заполнения): ')
        if date_contract == '':
            date_contract = datetime.today().strftime('%Y-%m-%d')
        more = []
        optional = create_opt_info(more)
        print (more)
        post_id = collection.insert_one({"name_client":name_client, "date_contract":date_contract}).inserted_id
        if more != None:
            for i in more:
                collection.update_one({'_id':post_id}, {'$set': i})
        print('done')

In [4]:
def to_read(coll):
    collection = db[coll]
    for c in collection.find():
            pprint.pprint(c)

In [5]:
def to_update(coll):
    try:
        collection = db[coll]
        id_to_up = input ('Введите ID необходмой записи: ')
        id_to_up = ObjectId(id_to_up)
        doc = [a for a in collection.find({'_id':id_to_up})][0]
        print (doc)
        param = input('Какой параметр вы хотите изменить (добавить)? ')
        text = input('Введите новое значение: ')
        collection.update_one({'_id':id_to_up}, {'$set': {param : text}})
        smth_else = input ('Что-то еще? (y/n) ')
        if smth_else == 'y':
            to_update(coll)
        else:
            print ('done')
    except:
        error = input ('Вы ввели несуществующий ID. Хотите продолжить? (y/n) ')
        if error == 'y':
            to_update(coll)

In [6]:
def to_delete(coll):
    collection = db[coll]
    id_to_del = input ('Введите ID необходмой записи: ')
    try:
        id_to_del = ObjectId(id_to_del)
        doc = [a for a in collection.find({'_id':id_to_del})][0]
        print (doc)
        yn = input('Хотите удалить этот документ? (y/n) ')
        if yn == 'y':
            collection.delete_one({'_id':id_to_del})
        smth_else = input ('Что-то еще? (y/n) ')
        if smth_else == 'y':
            to_delete(coll)
        else:
            print ('done')
    except:
        error = input ('Вы ввели несуществующий ID. Хотите продолжить? (y/n) ')
        if error == 'y':
            to_delete(coll)

In [7]:
def to_aggregate(coll):
    collection = db[coll]
    what = input ('Какой параметр вы хотите посчитать? ')
    pipeline = [{"$unwind": f"${what}"}, {"$group": {"_id": f"${what}", "count": {"$sum": 1}}}]
    counted = list(collection.aggregate(pipeline))
    if len(counted) != 0:
        for i in counted:
            print (str(i['_id']) + ' : ' + str(i['count']))
    else: 
        error = input ('Вы ввели несуществующий параметр. Хотите продолжить? (y/n) ')
        if error == 'y':
            to_aggregate(coll)

In [8]:
def to_sort(coll):
    rev = input ('От новых к старым или от старых к новым? (1/2)')
    collection = db[coll]
    dict_docs = {}
    if coll == 'projects':
        for c in collection.find():
            data = c['date']
            dict_docs[data] = c
    elif coll == 'clients':
        for c in collection.find():
            data = c['date_contract']
            dict_docs[data] = c
    if rev == '1':
        for elem in sorted(dict_docs.items(), reverse=True):
            print(elem[0] , "----" , elem[1] )
    elif rev == '2':
        for elem in sorted(dict_docs.items(), reverse=False):
            print(elem[0] , "----" , elem[1] )
    else: 
        error = input ('Вы ввели несуществующий параметр. Хотите продолжить? (y/n) ')
        if error == 'y':
            to_sort(coll)

In [9]:
def to_find(coll):
    if coll == 'clients':
        collection = db[coll]
        type_search = input ('Будем искать по названию компании. Вам известна первая буква или часть названия? (1/2) ')
        if type_search == '1':
            known = input ('Введите первую букву: ')
            for d in collection.find({"name_client":{"$regex":str(known)+'.*'}}):
                print (d)
        elif type_search == '2':
            known = input ('Введите часть: ')
            for d in collection.find({"name_client":{"$regex":'.*'+str(known)+'.*'}}):
                print (d)
        else:
            error = input ('Пожалуйста, выберите один из предложенных вариантов. Хотите продолжить? (y/n) ')
            if error == 'y':
                to_find(coll)
    elif coll == 'projects':
        collection = db[coll]
        type_search = input ('Будем искать по конкретному параметру или по всей коллекции? (1/2)' )
        if type_search == '1':
            known_param = input ('Введите параметр: ')
            known_part = input ('Введите искомое содержание: ')
            for d in collection.find({known_param:{"$regex":'.*'+str(known_part)+'.*'}}):
                print (d)
        elif type_search == '2':
            known_part = input ('Введите искомое содержание: ')
            result=[]
            for d in collection.find():
                for values in d:
                    for c in collection.find({values:{"$regex":'.*'+str(known_part)+'.*'}}):
                        if c in result:
                            pass
                        else:
                            result.append(c)
            for r in result:
                print (r)
        else:
            error = input ('Пожалуйста, выберите один из предложенных вариантов. Хотите продолжить? (y/n) ')
            if error == 'y':
                to_find(coll)

In [10]:
def first_step():
    coll = input('Что вас интересует? Введите "W" если свадебные проекты или "Q" если информация о клиентах: ')
    if coll == 'W':
        collection = 'projects'
    elif coll == 'Q':
        collection = 'clients'
    else:
        print ('Пожалуйста, выберите один из предложенных вариантов')
        first_step()
    to_do = input('Что вы хотите сделать? Введите "C" если создать, "R" если прочесть, "U" если обновить, "D" если удалить, \n "A" если посчитать, "S" если отсортировать по дате и "F" если искать: ')
    if to_do == 'C':
        to_create(collection)
    elif to_do == 'R':
        to_read(collection)
    elif to_do == 'U':
        to_update(collection)
    elif to_do == 'D':
        to_delete(collection)
    elif to_do == 'A':
        to_aggregate(collection)
    elif to_do == 'S':
        to_sort(collection)
    elif to_do == 'F':
        to_find(collection)
    else:
        print ('Пожалуйста, выберите один из предложенных вариантов')
        first_step()
    smth_else = input ('Что-то еще? (y/n) ')
    if smth_else == 'y':
        first_step()

In [16]:
first_step()

Что вас интересует? Введите "W" если свадебные проекты или "Q" если информация о клиентах: W
Что вы хотите сделать? Введите "C" если создать, "R" если прочесть, "U" если обновить, "D" если удалить, 
 "A" если посчитать и "S" если отсортировать по дате: U
Введите ID необходмой записи: 5e89a0a54ddf2cb75db776ca
Вы ввели несуществующий ID. Хотите продолжить? (y/n) n
Что-то еще? (y/n) y
Что вас интересует? Введите "W" если свадебные проекты или "Q" если информация о клиентах: W
Что вы хотите сделать? Введите "C" если создать, "R" если прочесть, "U" если обновить, "D" если удалить, 
 "A" если посчитать и "S" если отсортировать по дате: S
От новых к старым или от старых к новым? (1/2)2
2019-03-22 ---- {'_id': ObjectId('5e88e6352e4fec81a32bda01'), 'id_wedding': '346', 'date': '2019-03-22', 'type_work': 'corrections', 'client_comment': 'black screen'}
2020-03-26 ---- {'_id': ObjectId('5e88e4ef2e4fec81a32bda00'), 'id_wedding': '354', 'date': '2020-03-26', 'type_work': 'corrections', 'client_comm

In [13]:
first_step()

Что вас интересует? Введите "W" если свадебные проекты или "Q" если информация о клиентах: W
Что вы хотите сделать? Введите "C" если создать, "R" если прочесть, "U" если обновить, "D" если удалить, 
 "A" если посчитать, "S" если отсортировать по дате и "F" если искать: R
{'_id': ObjectId('5e88de5a2e4fec81a32bd9ff'),
 'client_comment': 'film 50 min',
 'date': '2020-04-02',
 'id_wedding': '354',
 'type_work': 'new_project'}
{'_id': ObjectId('5e88e4ef2e4fec81a32bda00'),
 'client_comment': 'more couple shots',
 'date': '2020-03-26',
 'id_wedding': '354',
 'type_work': 'corrections'}
{'_id': ObjectId('5e88e6352e4fec81a32bda01'),
 'client_comment': 'black screen',
 'date': '2019-03-22',
 'id_wedding': '346',
 'type_work': 'corrections'}
{'_id': ObjectId('5e89b2b54ddf2cb75db776cb'),
 'client_comment': 'couple shots are the best. please add more couple ',
 'date': '2020-04-05',
 'id_wedding': '350',
 'type_work': 'corrections'}
Что-то еще? (y/n) y
Что вас интересует? Введите "W" если свадебные